In [0]:
%run "../commons/configuration"

In [0]:
%run "../commons/common_functions"

In [0]:
dbutils.widgets.text("env_name","DEV")
p_env_name = dbutils.widgets.get("env_name")

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import to_timestamp, concat, lit, col

races_df = spark.read.csv(f"{raw_mount_path}/races.csv", 
                             header=True, 
                             inferSchema=True)\
                                .withColumnRenamed("raceId", "race_id")\
                                .withColumnRenamed("year", "race_year")\
                                .withColumnRenamed("circuitId", "circuit_id")\
                                .withColumn( "race_timestamp", to_timestamp(concat(col("date"), lit(" "), col("time")),"yyyy-MM-dd HH:mm:ss"))\
                                .drop("url")\
                                .withColumn("data_source",lit(p_env_name))

In [0]:
races_df_v2 = add_ingestion_date(races_df)
display(races_df_v2)

In [0]:
#races_df_v2.write.parquet(f"{processed_mount_path}/races", mode="overwrite", partitionBy=["race_year"])
races_df_v2.write.partitionBy("race_year").mode("overwrite").format("parquet").saveAsTable("processed.races")

In [0]:
dbutils.notebook.exit("Success")